In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
comments_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")
valid_df = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
sub_df=pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")

In [ ]:
prev_train_df = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
prev_train_df

In [ ]:
prev_train_df["severe_toxic"] = prev_train_df["severe_toxic"] * 2.5
prev_train_df["threat"] = prev_train_df["threat"] *3
prev_train_df["identity_hate"] = prev_train_df["identity_hate"] * 2

prev_train_df["total_toxic"] = prev_train_df[["toxic", "severe_toxic", "obscene", "threat", "insult",
                                              "identity_hate"]].sum(axis = 1).astype(np.int64)

In [ ]:
train_df = prev_train_df[["comment_text", "total_toxic"]]
train_df

In [ ]:
tfvec = TfidfVectorizer(analyzer = 'char_wb', ngram_range = (3,5))
tfv = tfvec.fit_transform(train_df["comment_text"])

In [ ]:
X=tfv[1:25001]
Y=train_df.loc[1:25000,['total_toxic']]

In [ ]:
reg = LinearRegression().fit(X,Y)

In [ ]:
tfv_comments = tfvec.transform(comments_df["text"])
pred = reg.predict(tfv_comments)


In [ ]:
a=pred[:1000]

In [ ]:
reg.score(X,Y) 

In [ ]:
sub = pd.DataFrame()
sub["comment_id"] = comments_df["comment_id"]
sub["score"] = pred
sub.to_csv('submission.csv',index=False)
sub

In [ ]:
import pickle
pickle.dump(reg,open('Model.pkl','wb'))